In [1]:
# Mount Google Drive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# wrap outputs cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
import os
import csv
import numpy as np
import pandas as pd
import sys
from datetime import datetime

In [4]:
# Data Path
root_dir = "gdrive/My Drive/Master_Thesis/"
test_dir = os.path.join(root_dir, 'data/Custom/T8-S10.csv')
data_dir = os.path.join(root_dir, 'predictions/ChatGPT-May-24-Version/')
save_dir = os.path.join(root_dir, 'predictions/')

In [5]:
df = pd.read_csv(test_dir)

In [6]:
def get_datetime(format):
    # datetime object containing current date and time
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt = now.strftime(format)
    return dt

In [7]:
# Read text file
import re

filenames = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
preds = None
for filename in filenames:
  with open(data_dir + filename, 'r') as f:
    print(filename)
    texts = f.read()
    texts = texts.split('------------------')
    texts = [text.replace('\n!\n', '') for text in texts]
    texts = [text.replace(
        '\nThis content may violate our content policy. If you believe this to be in error, please submit your feedback — your input will aid our research in this area.\n',
        '') for text in texts]
    texts = [re.sub("(Hateful message:).*(Counter speech:)", "", text, flags=re.DOTALL) for text in texts]
    texts = [re.sub("\*\*.*\*\*", "", text, flags=re.DOTALL) for text in texts]
    texts = [text.strip() for text in texts]
    print(texts[-1])
    initial = []
    response = []

    idx = 0
    while idx < len(texts):
      initial.append(texts[idx])
      response.append(texts[idx+1])
      idx += 2
    data = {'Hate_Speech': initial,
         'Prediction': response}
    preds = pd.DataFrame.from_dict(data)

    df_new = df.merge(preds, on="Hate_Speech", how = 'left')
    modelname = os.path.splitext(filename)[0]
    df_new.to_csv(save_dir + modelname + '_prediction_' + get_datetime("%d_%m_%Y__%H_%M_%S") + ".csv")

ChatGPT_3.5.txt
Promoting blanket statements about Islam or any religious group being incompatible with Western society is an oversimplification and can perpetuate harmful stereotypes. It is important to approach discussions about religion and cultural integration with nuance, empathy, and a commitment to fostering understanding.

Integration and cultural exchange are ongoing processes that require efforts from both host societies and individuals who are new to the country. It is crucial to recognize that individuals from diverse backgrounds can contribute positively to the social, cultural, and economic fabric of Western societies.

Rather than focusing on divisive narratives, let's work towards building inclusive societies that embrace diversity, promote equal opportunities, and foster dialogue and understanding between different cultures and religions. By fostering empathy, respect, and inclusivity, we can create a more harmonious world where everyone's rights and dignity are upheld

In [8]:
df_new

,Unnamed: 0,Dataset,Row ID,Hate_Speech,Counter_Speech,Target,Target_2,Prediction
0,2946,Multi_CONAN,1302,We allowed in thousands of people with serious...,Mental health services crisis is not to blame ...,DISABLED,NaN,While it's true that mental health services ar...
1,9036,Multi_CONAN,3304,Physically disabled folks deserve to be used.,It is important to acknowledge that there are ...,DISABLED,NaN,It's troubling to see such a negative portraya...
2,5273,Multi_CONAN,625,Downies are a problem: they create tensions wi...,"Did you know that people with Down syndrome, i...",DISABLED,NaN,It's essential to approach this topic with emp...
3,584,Multi_CONAN,3307,Physically disabled folks are all angry and us...,At least they are well aware of the negative e...,DISABLED,NaN,This statement is both harmful and untrue. Peo...
4,3690,Multi_CONAN,4723,Mentally disabled folks can't speak.,Mental health and disability are quite separat...,DISABLED,NaN,This statement is inaccurate and misleading. T...
...,...,...,...,...,...,...,...,...
75,3705,Multi_CONAN,4056,We must put an end to immigration. Islam is a ...,Immigration has been a huge part of Europe for...,other,NaN,It's crucial to differentiate between the conc...
76,2556,Multi_CONAN,723,Fat people are a burden to society because of ...,"The real problem is that, whether or not someo...",other,NaN,It's important to approach this topic with emp...
77,4327,Multi_CONAN,1881,We cannot allow Muslims into UK because it is ...,Muslims make up around 4 percent of the popula...,other,NaN,"The United Kingdom, like any other country, ha..."
78,4137,Multi_CONAN,515,Gypsies like living in trailer parks.,Only a very small portion lives in the trailer...,other,NaN,It's important to avoid stereotypes when discu...
